In [2]:
import argparse, os, re, json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

In [58]:
pred_st = '575'
type_t = 'upos_feats'
tb_v = '2.11'
tests_parsed_dir = '../../../../steps-parser/src/tests-parsed/'
conllu_list = os.listdir(tests_parsed_dir)
pred_filename = [f for f in conllu_list if f.startswith(pred_st)][0]
pred_filepath = os.path.join(tests_parsed_dir, pred_filename)
gold_filepath = '../../../tr_boun/v%s/test.conllu' % tb_v

gold_sent_pattern = r'# sent_id = (.*?)\n# text = (.*?)\n(.*?)\n\n'
pred_sent_pattern = r'(.*?)\n\n'
with open(gold_filepath, 'r', encoding='utf-8') as f:
    gold_tb = f.read()
with open(pred_filepath, 'r', encoding='utf-8') as f:
    pred_tb = f.read()
gold_sents = re.findall(gold_sent_pattern, gold_tb, re.DOTALL)
pred_sents = re.findall(pred_sent_pattern, pred_tb, re.DOTALL)

In [59]:
feat_d = dict()
for i in range(len(gold_sents)):
    g_sent_id, g_text, g_lines_str = gold_sents[i]
    p_lines_str = pred_sents[i]

    g_lines = g_lines_str.split('\n')
    p_lines = p_lines_str.split('\n')
    for j in range(len(g_lines)):
        g_feats = g_lines[j].split('\t')[5].split('|')
        p_feats = p_lines[j].split('\t')[5].split('|')
        g_feat_d = dict()
        p_feat_d = dict()
        for g_feat_t in g_feats:
            if g_feat_t == '_':
                continue
            g_tag_t, g_val_t = g_feat_t.split('=')
            g_feat_d[g_tag_t] = g_val_t
        for p_feat_t in p_feats:
            if p_feat_t == '_':
                continue
            p_tag_t, p_val_t = p_feat_t.split('=')
            p_feat_d[p_tag_t] = p_val_t
        all_feats_s = set(list(g_feat_d.keys()) + list(p_feat_d.keys()))

        for feat_t in all_feats_s:
            if feat_t not in feat_d.keys():
                feat_d[feat_t] = dict()
            if feat_t not in g_feat_d.keys():
                gold_t = '_'
            else:
                gold_t = g_feat_d[feat_t]
            if gold_t not in feat_d[feat_t].keys():
                feat_d[feat_t][gold_t] = dict()
            if feat_t not in p_feat_d.keys():
                pred_t = '_'
            else:
                pred_t = p_feat_d[feat_t]
            if pred_t not in feat_d[feat_t][gold_t].keys():
                feat_d[feat_t][gold_t][pred_t] = 0
            feat_d[feat_t][gold_t][pred_t] += 1

In [ ]:
feat_l = list(feat_d.keys())
col_cnt = 6
row_cnt = len(feat_l) // col_cnt + 1 if len(feat_l) % col_cnt != 0 else len(feat_l) // col_cnt
fig, axes = plt.subplots(row_cnt, col_cnt, figsize=(30, 30))
for idx, feat_t in enumerate(sorted(feat_l)):
    row_t = idx // col_cnt
    col_t = idx % col_cnt
    all_vals = set()
    val_l = list(feat_d[feat_t].keys())
    all_vals = all_vals.union(set(val_l))
    for gold_t in val_l:
        val_val_l = list(feat_d[feat_t][gold_t].keys())
        all_vals = all_vals.union(set(val_val_l))
    all_val_l = sorted(list(all_vals))
    board = [[0 for i in range(len(all_val_l))] for j in range(len(all_val_l))]
    for i, val1 in enumerate(all_val_l):
        for j, val2 in enumerate(all_val_l):
            if val1 not in feat_d[feat_t].keys():
                board[i][j] = 0
            elif val2 not in feat_d[feat_t][val1].keys():
                board[i][j] = 0
            else:
                board[i][j] = feat_d[feat_t][val1][val2]
    df_cm = pd.DataFrame(board, index=all_val_l, columns=all_val_l)
    axes[row_t, col_t].set_title(label=feat_t)
    sn.heatmap(df_cm, annot=True, cmap='OrRd', ax=axes[row_t, col_t], fmt='g')
plt.suptitle('%s CM for %s and %s' % (type_t, pred_st, tb_v))
# plt.tight_layout()
plt.savefig('%s/v%s-%s.png' % (type_t, tb_v, pred_st))